# Plotly Dataframe 

In [1]:
#import plotly.plotly as py
import plotly
import numpy as np
import pandas as pd
import plotly.plotly as py
import plotly.tools as tls
import plotly.graph_objs as go
import datetime
import time

tls.set_credentials_file(username='lxue10',api_key="F29SN8EVwOJKhZBoT3wG")


In [24]:
df = pd.read_csv('VPIN_new.csv',index_col = [0],parse_dates = True)

In [2]:
#stream_ids = tls.get_credentials_file()['stream_ids']
#print(stream_ids)
stream_ids = [u'8i780q45ku','tnc2io6pay','v2icvduevy']

# Get stream id from stream id list 
stream_id1 = stream_ids[0]
stream_id2 = stream_ids[1]

# Make instance of stream id object 
stream_1 = go.scatter.Stream(
    token=stream_id1,  # link stream id to 'token' key
    maxpoints=400      # keep a max of 80 pts on screen
)

stream_2 = go.scatter.Stream(
    token=stream_id2,  # link stream id to 'token' key
    maxpoints=400      # keep a max of 80 pts on screen
)
# input dataframe, draw the VPIN live plot
# Initialize trace of streaming plot by embedding the unique stream_id


In [25]:
# Initialize trace of streaming plot by embedding the unique stream_id
trace1 = go.Scatter(
    x=[],
    y=[],
    mode='lines+markers',
    stream=stream_1         # (!) embed stream id, 1 per trace
)

trace2 = go.Scatter(
    x=[],
    y=[],
    mode='lines+markers',
    stream=stream_2         # (!) embed stream id, 1 per trace
)


# Add title to layout object
#layout = go.Layout(title='Time Series')

# Make a figure object
fig = tls.make_subplots(rows=2, cols=1)
fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 2, 1)
#fig = go.Figure(data=data, layout=layout)
#fig = go.Figure(data = [trace1, trace2], layout = layout)
one_more_trace=dict(type='scatter',
                                   x=[df.index[0],df.index[-1]],
                                   y=[0.99,0.99],
                                   mode='lines',
                                   line=dict(color='red'))
fig.append_trace(one_more_trace, 1, 1)
# Send fig to Plotly, initialize streaming plot, open new tab
py.iplot(fig, filename='python-streaming')

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]



In [26]:
# We will provide the stream link object the same token that's associated with the trace we wish to stream to
s1 = py.Stream(stream_id1)
s2 = py.Stream(stream_id2)

# We then open a connection
s1.open()
s2.open()
# (*) Import module keep track and format current time


In [27]:
# Use plotly to draw two subplots with max VPIN, corresponding close price and threshold
def plotly_df(df):
    i = 0    # a counter

    # Delay start of stream by 5 sec (time to switch tabs)
    time.sleep(5)

    t = 0
    while (i<len(df)):

        x = df.time[i]
        y = df.VPIN[i]
        z = df['close price'][i]
        #y = float(np.random.randn(1))
        i += 1

        # Send data to your plot
        s1.write(dict(x=x, y=y))
        s2.write(dict(x=x, y=z))

        #     Write numbers to stream to append current data on plot,
        #     write lists to overwrite existing data on plot

        time.sleep(1)  # plot a point every second    
    # Close the stream when done plotting
    s1.close()
    s2.close()
    


In [28]:
ticktime = np.array(df.index) 
df['time'] = ticktime
df.index = range(len(df))
df_max = df.loc[df.groupby(by = 'time')['VPIN'].idxmax()]
df_max.index = range(len(df_max))
df_max #uniq max VPIN sub df 

,VPIN,close price,time
0,0.900410,28.770,2010-03-01 09:30:01
1,0.923326,28.775,2010-03-01 09:30:03
2,0.936909,28.765,2010-03-01 09:30:11
3,0.922469,28.765,2010-03-01 09:30:21
4,0.905441,28.770,2010-03-01 09:30:43
5,0.876876,28.765,2010-03-01 09:30:47
6,0.806924,28.730,2010-03-01 09:30:51
7,0.687992,28.720,2010-03-01 09:31:07
8,0.629904,28.720,2010-03-01 09:31:21
9,0.500201,28.730,2010-03-01 09:31:36


In [30]:
plotly_df(df_max)